In [4]:
import matplotlib.pyplot as plt
from itertools import combinations
from random import random,randint
import numpy as np
from functools import reduce
from scipy.optimize import minimize
from scipy.linalg import expm,sinm,cosm

import time
import math
from   matplotlib import cm
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute
from qiskit.quantum_info.analysis import average_data
from qiskit import *
from qiskit.quantum_info.operators import Operator
from qiskit.aqua.utils import tensorproduct
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor
from qiskit.visualization import plot_histogram
import picos as pic
import warnings
from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import CplexOptimizer
import scipy.linalg as la
from itertools import permutations, combinations


In [5]:
def num_1_pos(d):
    get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    num_1 = get_indexes(1,d)
    return num_1,len(num_1)

def num_0_pos(d):
    get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    num_0 = get_indexes(0,d)
    return num_0,len(num_0)

def column(matrix, i):
    return [row[i] for row in matrix]

#Construct initial state 
def init_s(n):
    x = pow(2,n)
    s = 1/np.sqrt(x) * np.ones(x)
    s = s.reshape(x, 1)
    return s

def bin2dec(string_num):
    return str(int(string_num, 2))

def row_binary(v):
    for i in range(len(v)):
        if v[i]%2 == 0:
            v[i] = 0
        else:
            v[i] = 1 
    return v

def converter(d):
    P = 0
    for i in range(len(d)):
        P+=d[i]*(2**(len(d)-i-1))
    return P
                 
#Define some pauli operators
PauliX = [[0,1],[1,0]]   
PauliZ = [[1,0],[0,-1]]
PauliI = np.eye(2)

#Define Cjk and B
def Cjk_generator(d):
    n = len(d)
    l = list(range(0,n))    
    for i in range(n):
        l[i] = PauliI
    s = num_1_pos(d)[0]
    for j in range(len(s)):
        l[s[j]] = PauliZ
    Cjk = l[0]
    for i in range(n-1):
        Cjk = np.kron(Cjk,l[i+1])
    return Cjk

def B_generator(n):
    l = list(range(0,n))
    b = list(range(0,n))
    
    for a in range(n):
        for i in range(n):
            l[i] = PauliI
        l[a]=PauliX
        temp = l[0]
        for i in range(n-1):
            temp = np.kron(temp,l[i+1])
        b[a] = temp
    B = sum(b[a] for a in range(n))
    return B

def basis(n):
    nums = np.array([a for a in range(2**(n))]) 
    bin_nums = (((nums.reshape(-1,1) & (2**np.arange(n)))) != 0).astype(int)
    a = bin_nums[:,::-1]
    return a


def matrix_binary(A):
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i][j]%2 == 0:
                A[i][j] = 0
            else:
                A[i][j] = 1
    return A

def error(n,err):
    e = np.zeros(n)
    a = np.random.randint(n,size=err)
    for i in range(len(a)):
        e[a[i]] = 1 
    return e
    




# Oringinal Generator matrix without error

In [6]:
G_oringin = np.array([[1,0,0,0,0,0,1,1,0,1],
                      [0,1,0,0,0,1,0,0,0,1],
                      [0,0,1,0,0,1,1,1,1,0],
                      [0,0,0,1,0,1,0,0,1,0],
                      [0,0,0,0,1,0,0,1,1,1]])
G_oringinT = G_oringin.T

In [21]:


n = G_oringin.shape[0]
u=0

for k in range(2**n):
    print("$$$$$$$$$$$$$$$$$$$")
    message = basis(n)[k]
    y = row_binary(message.dot(G_oringin))
    print("y = ",y)
    C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
    B = B_generator(G_oringinT.shape[1])
    s = init_s(n)
    s_d = (s).T
    print("the number of basis",k+1)

    #run the program
    start_time = time.time()
    step_size   = 0.05
    result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
    a_gamma         = np.arange(0, np.pi, step_size)
    a_beta          = np.arange(0, np.pi, step_size)
    #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
    f_p_max = 0
    f_p_max_gamma = 0
    f_p_max_beta = 0
    f_psum = 0

    for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
        for j in range(0, int(3.14/step_size)+1):       # j for a_beta

            #Construct U_C
            U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
            #Construct U_B
            U_B = expm(-1j*a_beta[j]*B)


            #Construct expection of each subgraph
            state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
            state_d = (state.conjugate()).T # bra


            f_p = (state_d.dot(C)).dot(state)
            f_psum += f_p


            if f_p.real > f_p_max:
                f_p_max = f_p.real
                f_p_max_gamma = a_gamma[i]
                f_p_max_beta = a_beta[j]

            result_array[i][j] = f_p.real
    print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
    print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
    print("\n Optimal gamma : ",f_p_max_gamma)
    print("\n Optimal beta : ",f_p_max_beta)


    end_time = time.time()
    #print("\n Runningtime : ",end_time - start_time)
    c = converter(message)
    r = np.ones(2**len(message))
    #Construct U_C
    U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
    #Construct U_B
    U_B = expm(-1j*f_p_max_beta*B)


    #Construct expection of each subgraph
    state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
    prob=[]
    for i in range(len(state)):
        p = (state[i].real)**2 + (state[i].imag)**2
        np.array(prob.append(float(p)))      
    e = np.max(prob)
    print(e)
    print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
    if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
        print("CANNOT correct！！！！！！")
    else:
        print("Good Job!!!!!!")
        u+=1

print("---------------")
print('correct rate: ',u/(2**n)*100,'%')



$$$$$$$$$$$$$$$$$$$
y =  [0 0 0 0 0 0 0 0 0 0]
the number of basis 1

 Average expectation using QAOA with fine grid :  [[4.61100818e-06]]

 Maximum expectation using QAOA with fine grid :  [[2.80080897]]

 Optimal gamma :  1.7000000000000002

 Optimal beta :  0.35000000000000003
5.895484627518482
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [0 0 0 0 1 0 0 1 1 1]
the number of basis 2

 Average expectation using QAOA with fine grid :  [[4.61100818e-06]]

 Maximum expectation using QAOA with fine grid :  [[2.80080897]]

 Optimal gamma :  1.7000000000000002

 Optimal beta :  0.35000000000000003
5.895484627518485
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [0 0 0 1 0 1 0 0 1 0]
the number of basis 3

 Average expectation using QAOA with fine grid :  [[4.61100818e-06]]

 Maximum expectation using QAOA with fine grid :  [[2.80080897]]

 Optimal gamma :  1.7000000000000002

 Optimal beta :  0.35000000000000003
5.895484627518485
most likey basis:   3
Good Jo


 Average expectation using QAOA with fine grid :  [[4.61100818e-06]]

 Maximum expectation using QAOA with fine grid :  [[2.80080897]]

 Optimal gamma :  1.7000000000000002

 Optimal beta :  0.35000000000000003
5.895484627518485
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [1 1 0 1 0 0 1 1 1 0]
the number of basis 27

 Average expectation using QAOA with fine grid :  [[4.61100818e-06]]

 Maximum expectation using QAOA with fine grid :  [[2.80080897]]

 Optimal gamma :  1.7000000000000002

 Optimal beta :  0.35000000000000003
5.895484627518488
most likey basis:   27
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [1 1 0 1 1 0 1 0 0 1]
the number of basis 28

 Average expectation using QAOA with fine grid :  [[4.61100818e-06]]

 Maximum expectation using QAOA with fine grid :  [[2.80080897]]

 Optimal gamma :  1.7000000000000002

 Optimal beta :  0.35000000000000003
5.895484627518485
most likey basis:   28
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [1 1 1 0 0 0 0 0 1 0]
the numbe

# Oringinal with one bit error

# Generate one bit error pattern

In [22]:
w1 = []
for i in range(G_oringin.shape[1]):
    o = np.zeros(G_oringin.shape[1])
    o[i]=1
    w1.append(o)      

In [24]:
G_oringinT = G_oringin.T
cor_rate=[]
n = G_oringin.shape[0]

for m in range(len(w1)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w1[m] 
        print("w = ",w)
        y = row_binary(message.dot(G_oringin)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
        B = B_generator(G_oringinT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
        if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
            print("CANNOT correct！！！！！！")
        else:
            print("Good Job!!!!!!")
            u+=1

    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)


$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 0. 0. 1. 1. 1.]
the number of basis 2
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 1. 0. 0. 1. 0.]
the number of basis 3
most likey basis:   3
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 1. 1. 0. 1. 0. 1.]
the number of basis 4
most likey basis:   4
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 0. 1. 1. 1. 1. 0.]
the number of basis 5
most likey basis:   5
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 1. 1. 1. 0. 0. 1.]
the number of basis 6
most likey basis:   6
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 0. 1

most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
the number of basis 25
most likey basis:   25
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 1. 1. 0. 1. 1.]
the number of basis 26
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 0. 1. 1. 1. 0.]
the number of basis 27
most likey basis:   27
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 1. 0. 1. 0. 0. 1.]
the number of basis 28
most likey basis:   28
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 

most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 0. 1. 0. 0. 1. 0. 0.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 0. 0. 1. 1. 1. 1.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 1. 0. 1. 0. 0. 0.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 1. 0. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 1. 0. 1. 0.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 0. 1. 1. 0. 1.]
the number of basis 17
most likey basis:   17
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 

most likey basis:   32
Good Job!!!!!!
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 1. 1. 0. 1. 1. 1.]
the number of basis 2
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
the number of basis 3
most likey basis:   3
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 1. 0. 0. 1. 0. 1.]
the number of basis 4
most likey basis:   4
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 0. 1. 0. 0. 0. 1. 1. 1. 0.]
the number of basis 5
most likey basis:   5
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
the number of basis 6
most likey basis:   6
Good Job!!!!!!
$$$$$

most likey basis:   21
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 0. 1. 0. 1. 1. 1. 1. 0. 0.]
the number of basis 22
most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 0. 1. 0. 0. 1.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 0. 1. 1. 1. 0. 1. 1. 1. 0.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 1. 0. 1. 0. 0.]
the number of basis 25
most likey basis:   25
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 26
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 0. 1. 1. 0.]
the number of basis 27
most likey basis:   27
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 

most likey basis:   10
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
the number of basis 11
most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 0. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 1. 0. 0. 0. 1. 1. 0. 1.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 1. 0. 1. 0. 1. 0. 1. 0.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 

most likey basis:   31
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 1. 1. 1. 1. 1. 0. 1. 1. 0.]
the number of basis 32
most likey basis:   32
Good Job!!!!!!
---------------
correct rate:  100.0 %
----------------------
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


# Oringinal with two bit error

In [7]:
w2 = []
comb = combinations([a for a in range(G_oringin.shape[1])], 2) 
comb = list(comb)
for a in range(len(comb)):
    o = np.zeros(G_oringin.shape[1])
    j = comb[a][0]
    k = comb[a][1]
    o[j]=1
    o[k]=1
    w2.append(o)
print(w2)   
    


[array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]), array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 1., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0., 1., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]), array([0., 0., 1., 1., 0., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 1., 

In [8]:

G_oringinT = G_oringin.T
cor_rate=[]
n = G_oringin.shape[0]

for m in range(len(w2)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w2[m] 
        print("w = ",w)
        y = row_binary(message.dot(G_oringin)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
        B = B_generator(G_oringinT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
        if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
            print("CANNOT correct！！！！！！")
        else:
            print("Good Job!!!!!!")
            u+=1

    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)




$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   29
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 0. 0. 1. 1. 1.]
the number of basis 2
most likey basis:   30
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 1. 0. 0. 1. 0.]
the number of basis 3
most likey basis:   31
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 1. 0. 1. 0. 1.]
the number of basis 4
most likey basis:   32
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 1. 1. 1. 1. 0.]
the number of basis 5
most likey basis:   25
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 1. 1. 0. 0. 1.]
the number of basis 6
most likey basis:   26
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0.

most likey basis:   20
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 1.]
the number of basis 21
most likey basis:   21
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 1. 1. 0. 1. 0. 0.]
the number of basis 22
most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 1. 0. 0.]
the number of basis 25
most likey basis:   25
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 1. 0. 1. 1.]
the number of basis 26
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 

most likey basis:   23
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 1. 0. 0. 0. 1.]
the number of basis 9
most likey basis:   26
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 10
most likey basis:   25
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 0. 0. 0. 1. 1.]
the number of basis 11
most likey basis:   28
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 0. 1. 0. 0.]
the number of basis 12
most likey basis:   27
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 0. 1. 1. 1. 1.]
the number of basis 13
most likey basis:   30
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 0. 1. 0. 0. 0.]
the number of basis 14
most likey basis:   29
CANNOT corre

most likey basis:   28
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 0. 1. 0. 0. 1. 0.]
the number of basis 29
most likey basis:   29
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]
the number of basis 30
most likey basis:   30
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
the number of basis 31
most likey basis:   31
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
the number of basis 32
most likey basis:   32
Good Job!!!!!!
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
the number of basis 1
most likey basis:   17
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
the number of basis 2
most likey basis:   18
CANN

most likey basis:   16
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
the number of basis 17
most likey basis:   17
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 0. 1. 0. 1. 1. 1. 0.]
the number of basis 18
most likey basis:   18
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 1. 0. 1. 1. 0. 1. 1.]
the number of basis 19
most likey basis:   19
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 1. 1. 1. 1. 1. 0. 0.]
the number of basis 20
most likey basis:   20
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 1. 0. 0. 1. 0. 1. 1. 1.]
the number of basis 21
most likey basis:   21
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 1. 0. 1. 1. 0. 0. 0. 0.]
the number of basis 22
most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 

...

most likey basis:   9
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
y =  [0. 1. 0. 1. 1. 1. 0. 1. 1. 0.]
the number of basis 12
most likey basis:   10
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 1. 0. 1.]
the number of basis 13
most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
y =  [0. 1. 1. 0. 1. 1. 1. 0. 1. 0.]
the number of basis 14
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
y =  [0. 1. 1. 1. 0. 0. 1. 1. 1. 1.]
the number of basis 15
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
y =  [0. 1. 1. 1. 1. 0. 1. 0. 0. 0.]
the number of basis 16
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
y =  [1. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
the number of basis 17
most likey basis:   19
CANNOT corre

most likey basis:   22
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
y =  [1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
the number of basis 31
most likey basis:   23
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
y =  [1. 1. 1. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 32
most likey basis:   24
CANNOT correct！！！！！！
---------------
correct rate:  0.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
the number of basis 1
most likey basis:   7
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
the number of basis 2
most likey basis:   8
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 0. 0. 1. 0. 1. 1. 1. 1. 0.]
the number of basis 3
most likey basis:   5
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 0. 0. 1. 1. 1. 1. 0. 0. 1.]
the number of basis 4
most

most likey basis:   18
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 19
most likey basis:   19
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 0. 1. 1. 1. 0. 0. 1. 0.]
the number of basis 20
most likey basis:   20
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 21
most likey basis:   21
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 1. 0. 1. 1. 1. 1. 1. 0.]
the number of basis 22
most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 1. 1. 0. 0. 1. 0. 1. 1.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 1. 1. 1. 0. 1. 1. 0. 0.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 

most likey basis:   6
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 0. 1. 1. 0. 0. 1. 0. 1. 0.]
the number of basis 7
most likey basis:   7
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 0. 1. 1. 1. 0. 1. 1. 0. 1.]
the number of basis 8
most likey basis:   8
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 0. 0. 0. 1. 0. 1. 1. 1.]
the number of basis 9
most likey basis:   9
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 0. 0. 1. 1. 0. 0. 0. 0.]
the number of basis 10
most likey basis:   10
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 0. 1. 0. 0. 0. 1. 0. 1.]
the number of basis 11
most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 0. 1. 1. 0. 0. 0. 1. 0.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0

most likey basis:   27
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [1. 1. 0. 1. 1. 0. 1. 1. 0. 0.]
the number of basis 28
most likey basis:   28
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [1. 1. 1. 0. 0. 0. 0. 1. 1. 1.]
the number of basis 29
most likey basis:   29
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]
the number of basis 30
most likey basis:   30
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [1. 1. 1. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 31
most likey basis:   31
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [1. 1. 1. 1. 1. 1. 0. 0. 1. 0.]
the number of basis 32
most likey basis:   32
Good Job!!!!!!
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
y =  [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
the number of basis 1
most likey basis:   1
Good Job!!

# Apply a full rank matrix on G

In [19]:
G_oringin = np.array([[1,0,0,1,0,1],
                      [0,1,0,1,1,1],
                      [0,0,1,1,1,0]])
P = np.array([[1,1,0],
              [1,0,1],
              [0,1,0]])
print(np.linalg.matrix_rank(P))
G = matrix_binary(P.dot(G_oringin))
print(G)

3
[[1 1 0 0 1 0]
 [1 0 1 0 1 1]
 [0 1 0 1 1 1]]


# Without error

In [18]:
GT = G.T

n = G.shape[0]
u=0

for k in range(2**n):
    print("$$$$$$$$$$$$$$$$$$$")
    message = basis(n)[k]
    y = row_binary(message.dot(G))
    print("y = ",y)
    C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
    B = B_generator(GT.shape[1])
    s = init_s(n)
    s_d = (s).T
    print("the number of basis",k+1)

    #run the program
    start_time = time.time()
    step_size   = 0.05
    result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
    a_gamma         = np.arange(0, np.pi, step_size)
    a_beta          = np.arange(0, np.pi, step_size)
    print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
    f_p_max = 0
    f_p_max_gamma = 0
    f_p_max_beta = 0
    f_psum = 0

    for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
        for j in range(0, int(3.14/step_size)+1):       # j for a_beta

            #Construct U_C
            U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
            #Construct U_B
            U_B = expm(-1j*a_beta[j]*B)


            #Construct expection of each subgraph
            state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
            state_d = (state.conjugate()).T # bra


            f_p = (state_d.dot(C)).dot(state)
            f_psum += f_p


            if f_p.real > f_p_max:
                f_p_max = f_p.real
                f_p_max_gamma = a_gamma[i]
                f_p_max_beta = a_beta[j]

            result_array[i][j] = f_p.real
    print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
    print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
    print("\n Optimal gamma : ",f_p_max_gamma)
    print("\n Optimal beta : ",f_p_max_beta)


    end_time = time.time()
    print("\n Runningtime : ",end_time - start_time)
    c = converter(message)
    r = np.ones(2**len(message))
    print(r)
    #Construct U_C
    U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
    #Construct U_B
    U_B = expm(-1j*f_p_max_beta*B)


    #Construct expection of each subgraph
    state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
    print(state)
    prob=[]
    for i in range(len(state)):
        p = (state[i].real)**2 + (state[i].imag)**2
        np.array(prob.append(float(p)))
    print(prob)      
    e = np.max(prob)
    print(e)
    print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
    if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
        print("CANNOT correct！！！！！！")
    else:
        print("Good Job!!!!!!")
        u+=1

print("---------------")
print('correct rate: ',u/(2**n)*100,'%')

$$$$$$$$$$$$$$$$$$$
y =  [0 0 0 0 0 0]
the number of basis 1
Numbers of iteration:  3969

 Average expectation using QAOA with fine grid :  [[1.24872842]]

 Maximum expectation using QAOA with fine grid :  [[5.38436809]]

 Optimal gamma :  1.4000000000000001

 Optimal beta :  1.0

 Runningtime :  3.584260940551758
[1. 1. 1. 1. 1. 1. 1. 1.]
[[-1.81842299-0.57479379j]
 [-0.28235308+0.17477034j]
 [ 0.12757721-0.35515928j]
 [-0.28235308+0.17477034j]
 [-0.28235308+0.17477034j]
 [ 0.12757721-0.35515928j]
 [-0.28235308+0.17477034j]
 [-1.81842299-0.57479379j]]
[3.6370500774147447, 0.11026793279691731, 0.1424140569914201, 0.1102679327969174, 0.11026793279691748, 0.14241405699142015, 0.11026793279691721, 3.6370500774147465]
3.6370500774147465
most likey basis:   8
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
y =  [0 1 1 0 0 1]
the number of basis 2
Numbers of iteration:  3969

 Average expectation using QAOA with fine grid :  [[1.24872842]]

 Maximum expectation using QAOA with fine grid :  [[5.3843

# One bit error decode

In [65]:
GT = G.T

n = G.shape[0]
cor_rate=[]
for m in range(len(w1)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w1[m] 
        print("w = ",w)
        y = row_binary(message.dot(G)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
        B = B_generator(GT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
        if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
            print("CANNOT correct！！！！！！")
        else:
            print("Good Job!!!!!!")
            u+=1

    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)





$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 1. 0. 0.]
the number of basis 2
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 1.]
the number of basis 3
most likey basis:   3
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 0. 1. 0. 1.]
the number of basis 4
most likey basis:   4
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 1. 0.]
the number of basis 5
most likey basis:   5
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 0. 1. 0.]
the number of basis 6
most likey basis:   6
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 1. 1. 1.]
the number of basis 7
most likey basis:   7
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0.]
y =  [0

most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 0.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0.]
y =  [0. 0. 1. 0. 1. 0. 0.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 0. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0.]
y =  [0. 1. 0. 1. 1. 0. 1.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 0. 1. 0. 0.]
the number of basis 1
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 0.]
the number of basis 2
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 1. 1. 1. 0. 1.]
the number of basis 3
most li

# Two bits error decode

In [66]:
GT = G.T

n = G.shape[0]
u=0
cor_rate=[]
for m in range(len(w2)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w2[m] 
        print("w = ",w)
        y = row_binary(message.dot(G)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
        B = B_generator(GT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
        if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
            print("CANNOT correct！！！！！！")
        else:
            print("Good Job!!!!!!")
            u+=1

    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)

$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 1. 0. 0.]
the number of basis 2
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 0. 1.]
the number of basis 3
most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 0. 1.]
the number of basis 4
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 1. 1. 0.]
the number of basis 5
most likey basis:   9
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 0. 1. 0.]
the number of basis 6
most likey basis:   10
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 1. 1.]
the number of basis 7
most likey basis:   11
CANNOT correct！！！！！！
$$$$$$$

most likey basis:   2
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 1. 1. 0. 1. 1.]
the number of basis 11
most likey basis:   3
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0.]
y =  [1. 0. 1. 0. 1. 1. 1.]
the number of basis 12
most likey basis:   4
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 1. 0. 1. 0. 0.]
the number of basis 13
most likey basis:   5
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0.]
y =  [1. 0. 1. 1. 0. 0. 0.]
the number of basis 14
most likey basis:   6
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 1. 1. 0. 1.]
the number of basis 15
most likey basis:   7
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 0. 0.]
y =  [1. 1. 0. 0. 0. 0. 1.]
the number of basis 16
most likey basis:   8
CANNOT correct！！！！！！
---------------
correct rate:  0.0 %
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0.]
y =  [1. 0. 0. 0. 0.

most likey basis:   12
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [1. 1. 1. 1. 1. 0. 1.]
the number of basis 4
most likey basis:   11
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [0. 0. 1. 1. 1. 1. 0.]
the number of basis 5
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 1. 0.]
the number of basis 6
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [0. 1. 0. 0. 1. 1. 1.]
the number of basis 7
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 1. 1.]
the number of basis 8
most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 1. 0.]
the number of basis 9
most likey basis:   2
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 0.]
the number of basis 10
most l

most likey basis:   5
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 1.]
y =  [1. 0. 1. 0. 0. 0. 1.]
the number of basis 13
most likey basis:   4
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 1.]
y =  [0. 1. 1. 1. 1. 0. 1.]
the number of basis 14
most likey basis:   3
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 1.]
y =  [1. 1. 0. 1. 0. 0. 0.]
the number of basis 15
most likey basis:   2
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 1.]
y =  [0. 0. 0. 0. 1. 0. 0.]
the number of basis 16
most likey basis:   1
CANNOT correct！！！！！！
---------------
correct rate:  0.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 1. 0. 0. 0.]
y =  [0. 0. 1. 1. 0. 0. 0.]
the number of basis 1
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 1. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 0. 0.]
the number of basis 2
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 1. 0. 0. 0.]
y =  [0. 1. 0. 0. 0.

most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [1. 0. 0. 1. 0. 1. 1.]
the number of basis 6
most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [0. 0. 1. 1. 1. 1. 0.]
the number of basis 7
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [1. 1. 1. 0. 0. 1. 0.]
the number of basis 8
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [1. 0. 1. 0. 1. 1. 1.]
the number of basis 9
most likey basis:   4
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [0. 1. 1. 1. 0. 1. 1.]
the number of basis 10
most likey basis:   3
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [1. 1. 0. 1. 1. 1. 0.]
the number of basis 11
most likey basis:   2
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 1.]
y =  [0. 0. 0. 0. 0. 1. 0.]
the number of basis 12
most l

most likey basis:   4
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 1.]
y =  [1. 0. 0. 0. 0. 0. 0.]
the number of basis 15
most likey basis:   1
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 1.]
y =  [0. 1. 0. 1. 1. 0. 0.]
the number of basis 16
most likey basis:   2
CANNOT correct！！！！！！
---------------
correct rate:  0.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 1. 0.]
y =  [0. 0. 0. 0. 1. 1. 0.]
the number of basis 1
most likey basis:   9
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 1. 0.]
y =  [1. 1. 0. 1. 0. 1. 0.]
the number of basis 2
most likey basis:   10
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 1. 1. 1. 1. 1.]
the number of basis 3
most likey basis:   11
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 1. 0.]
y =  [1. 0. 1. 0. 0. 1. 1.]
the number of basis 4
most likey basis:   12
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 1. 0. 0. 

# Three bits error decode

In [37]:
GT = G.T

n = G.shape[0]
u=0

for k in range(2**n):
    print("$$$$$$$$$$$$$$$$$$$")
    message = basis(n)[k]
    w = error(G.shape[1],3) 
    print("w = ",w)
    y = row_binary(message.dot(G)+w)
    print("y = ",y)
    C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
    B = B_generator(GT.shape[1])
    s = init_s(n)
    s_d = (s).T
    print("the number of basis",k+1)

    #run the program
    start_time = time.time()
    step_size   = 0.05
    result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
    a_gamma         = np.arange(0, np.pi, step_size)
    a_beta          = np.arange(0, np.pi, step_size)
    print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
    f_p_max = 0
    f_p_max_gamma = 0
    f_p_max_beta = 0
    f_psum = 0

    for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
        for j in range(0, int(3.14/step_size)+1):       # j for a_beta

            #Construct U_C
            U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
            #Construct U_B
            U_B = expm(-1j*a_beta[j]*B)


            #Construct expection of each subgraph
            state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
            state_d = (state.conjugate()).T # bra


            f_p = (state_d.dot(C)).dot(state)
            f_psum += f_p


            if f_p.real > f_p_max:
                f_p_max = f_p.real
                f_p_max_gamma = a_gamma[i]
                f_p_max_beta = a_beta[j]

            result_array[i][j] = f_p.real
    print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
    print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
    print("\n Optimal gamma : ",f_p_max_gamma)
    print("\n Optimal beta : ",f_p_max_beta)


    end_time = time.time()
    print("\n Runningtime : ",end_time - start_time)
    c = converter(message)
    r = np.ones(2**len(message))
    print(r)
    #Construct U_C
    U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
    #Construct U_B
    U_B = expm(-1j*f_p_max_beta*B)


    #Construct expection of each subgraph
    state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
    print(state)
    prob=[]
    for i in range(len(state)):
        p = (state[i].real)**2 + (state[i].imag)**2
        np.array(prob.append(float(p)))
    print(prob)      
    e = np.max(prob)
    print(e)
    print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
    if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
        print("CANNOT correct！！！！！！")
    else:
        print("Good Job!!!!!!")
        u+=1
print("---------------")
print('correct rate: ',u/16*100,'%')

$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 1. 1. 0.]
y =  [1. 0. 0. 0. 1. 1. 0.]
the number of basis 1
Numbers of iteration:  3969

 Average expectation using QAOA with fine grid :  [[3.47263841e-07]]

 Maximum expectation using QAOA with fine grid :  [[1.72303944]]

 Optimal gamma :  1.4500000000000002

 Optimal beta :  2.8000000000000003

 Runningtime :  4.77441930770874
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[-0.19710923-0.57855201j]
 [-0.41449365-1.18998994j]
 [ 0.55409536-0.09622739j]
 [-0.58597445-0.9229143j ]
 [-0.41449365-0.71149128j]
 [-0.19710923-0.78877793j]
 [-0.58597445-0.44441564j]
 [-0.29206259-0.84974447j]
 [ 0.55409536-1.80525383j]
 [ 0.19175599-0.44441564j]
 [-0.19710923-0.10005335j]
 [-0.41449365-0.71149128j]
 [ 0.19175599-0.9229143j ]
 [-0.29206259-1.05173676j]
 [-0.41449365-1.18998994j]
 [-0.19710923-1.26727659j]]
[0.37357447730583887, 1.5878810479791676, 0.31628137760799724, 1.1951368546995627, 0.6780248339849377, 0.6610226711733735, 0.5408713126788904, 0.8


 Average expectation using QAOA with fine grid :  [[3.47263743e-07]]

 Maximum expectation using QAOA with fine grid :  [[1.67835972]]

 Optimal gamma :  1.4500000000000002

 Optimal beta :  2.8000000000000003

 Runningtime :  4.598299026489258
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[ 0.11798608-1.33345536j]
 [ 0.19643478-1.11280241j]
 [-0.30790443-1.23707431j]
 [ 0.46752609+0.08510129j]
 [ 0.02401755-1.07796416j]
 [-0.05443115-1.0500733j ]
 [-0.01024878-1.22938094j]
 [-0.48032165-0.95369226j]
 [ 0.13123351-0.11362448j]
 [-0.10201439-1.62856218j]
 [-0.4515544 -0.21000553j]
 [-0.37310569+0.01064742j]
 [-0.57978926-0.59475564j]
 [-0.04118372-0.67768223j]
 [-0.54552292-0.80195413j]
 [-0.62397163-0.77406328j]]
[1.7920239095981059, 1.2769158317341918, 1.6251579910665024, 0.22582287300999138, 1.1625835706884848, 1.1056166938474417, 1.511482527069244, 1.1402378138036244, 0.03013275816711869, 2.6626217123838423, 0.24800369761799632, 0.13932122642225012, 0.6898898593760219, 0.46094


 Average expectation using QAOA with fine grid :  [[3.46702323e-07]]

 Maximum expectation using QAOA with fine grid :  [[2.07559769]]

 Optimal gamma :  3.0

 Optimal beta :  2.8000000000000003

 Runningtime :  5.416161060333252
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[ 0.86330246+3.90074932e-01j]
 [-0.33388663+4.88350065e-04j]
 [ 0.41524305+5.13956890e-01j]
 [ 0.01455658+1.26074662e+00j]
 [ 0.05860031+2.37532727e-01j]
 [ 0.86330246+9.48905928e-01j]
 [ 0.01455658+7.82342406e-02j]
 [ 0.41524305+1.07278789e+00j]
 [ 0.41524305+1.07278789e+00j]
 [ 0.01455658-2.41227874e-01j]
 [-0.23448214+9.48905928e-01j]
 [-0.33388663+1.58625643e+00j]
 [ 0.01455658+1.58020873e+00j]
 [ 0.41524305+5.13956890e-01j]
 [ 0.05860031+1.34921205e+00j]
 [-0.23448214+3.90074932e-01j]]
[0.8974495865604661, 0.11148052250731276, 0.43657847290614293, 1.589693933780702, 0.05985579328608538, 1.6457135950137112, 0.00633249047047108, 1.3233006369843572, 1.3233006369843578, 0.05840278106867861, 0.955404334050066